# Before Building Model

In [ ]:
from IPython.display import clear_output
!pip3 install flaml 
clear_output()

import pandas as pd
import numpy as np
import os
import random
import warnings
warnings.simplefilter('ignore')

from flaml import AutoML

TRAIN_PATH = "../input/house-prices-advanced-regression-techniques/train.csv"
TEST_PATH = "../input/house-prices-advanced-regression-techniques/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/house-prices-advanced-regression-techniques/sample_submission.csv"
SUBMISSION_PATH = "submission.csv"

ID = "Id"
TARGET = "SalePrice"

SEED = 2022
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything()

MODEL_TIME_BUDGET = 60
MODEL_METRIC = 'rmse'
MODEL_TASK = "regression"
MODEL_LIST = ['lgbm', 'catboost', 'xgboost']
MODEL_LOG_FILE_PATH = "flaml_log.log"

CROSS_VAL_CV = 3

In [ ]:
from shutil import copyfile
copyfile(src = "../input/auto-label-encoding-util/auto_label_encoding_process.py", dst = "../working/auto_label_encoding_process.py")
from auto_label_encoding_process import *

train,test = autoLabelEncodingByPath(TRAIN_PATH,TEST_PATH,TARGET)

# Build Model

In [ ]:
X = train.drop([ID,TARGET],axis=1)
y = train[TARGET]

model = AutoML()
params = {
    "time_budget": MODEL_TIME_BUDGET,  
    "metric": MODEL_METRIC,
    "estimator_list": MODEL_LIST, 
    "task": MODEL_TASK,
    "seed":SEED,
    "log_file_name":MODEL_LOG_FILE_PATH,
}
model.fit(X, y, **params)

In [ ]:
model.model.model

In [ ]:
from sklearn.model_selection import cross_val_score
neg_score = cross_val_score(model.model.model, X, y, cv=CROSS_VAL_CV,scoring='neg_mean_squared_error')
score = np.sqrt(-neg_score)
score

# After Building Model

In [ ]:
sub = pd.read_csv(SAMPLE_SUBMISSION_PATH)
sub[TARGET] = model.predict(test)
sub.to_csv(SUBMISSION_PATH,index=False)
sub.head()